In [49]:
import requests

def display_webpage_content(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # Send a GET request to the specified URL with the custom User-Agent
        response = requests.get(url, headers=headers)
        
        # Check if the request was successful
        if response.status_code == 200:
            print("Page loaded successfully!")
            print("Response Content:")
            # print(response.text)  # Print the entire response content
            last_line = response.text.strip().splitlines()[-1]
            print(last_line)
        else:
            print(f"Failed to load the page. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return last_line

# Example usage
url = "https://crex.live/series/1MV/big-bash-league-2024-25/team-squad"
last_line = display_webpage_content(url)

Page loaded successfully!
Response Content:
<script type="application/ld+json" id="webpage-schema">{"@context":"https://schema.org","@type":"WebPage","url":"https://crex.live/series/1MV/big-bash-league-2024-25/team-squad","name":"BBL 2024-25 Teams, Squads and Players List - Big Bash League 2024-25","headline":"BBL 2024-25 Teams, Squads and Players List - Big Bash League 2024-25","description":"Get list of BBL 2024-25 teams, squads and players playing in BBL 2024-25. Catch up with detailed list of batsmen, bowler and allrounders in BBL 2024-25.","inLanguage":"English","publisher":{"@type":"Organization","name":"CREX","url":"https://crex.live","logo":{"@type":"ImageObject","url":"https://crex.live/assets/logo/logo2.webp","width":"60","height":"60"}},"mainEntity":{"@type":"Thing","name":"BBL 2024-25 Teams, Squads and Players List - Big Bash League 2024-25"}}</script><script id="app-root-state" type="application/json">{&q;https://crickapi.com/turtle/home/getHomeSeries&q;:[{&q;id&q;:2236,&q

In [50]:
import json
import re
import csv
from io import StringIO

def extract_bbl_2013_14_data(input_text):
    # Find the script content with id="app-root-state"
    script_match = re.search(r'<script id="app-root-state" type="application/json">(.*?)</script>', input_text, re.DOTALL)
    if not script_match:
        print("No script data found.")
        return

    # Extract JSON content and clean it
    json_str = script_match.group(1)
    json_str = json_str.replace('&q;', '"').replace('&quot;', '"')  # Replace encoded quotes

    try:
        data = json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return

    # Extract player and team mappings from "https://oc.crickapi.com/mapping/getHomeMapDataseriesmain"
    main_data = data.get("https://oc.crickapi.com/mapping/getHomeMapDataseriesmain", {})
    players = {p["f_key"]: p["n"] for p in main_data.get("p", [])}
    teams = {t["f_key"]: {"name": t["n"], "short_name": t["sn"]} for t in main_data.get("t", [])}

    # Extract squad data from "https://crickapi.com/series/getSeriesOverview"
    series_data = data.get("https://crickapi.com/series/getSeriesOverview", {}).get("i1", {}).get("s", [])
    team_players = {}
    for squad in series_data:
        team_fkey = squad["tf"]
        if team_fkey in teams:
            # Extract player f_keys from "pf" (player field)
            player_fkeys = [pf.split("/")[0] for pf in squad.get("pf", [])]
            team_players[team_fkey] = player_fkeys

    # Build the CSV content
    csv_output = StringIO()
    csv_writer = csv.writer(csv_output)
    csv_writer.writerow(["Player Name", "Team Name", "Team Short Name"])

    for team_fkey, player_fkeys in team_players.items():
        team_info = teams.get(team_fkey, {"name": "Unknown", "short_name": "UNK"})
        for player_fkey in player_fkeys:
            player_name = players.get(player_fkey, "Unknown Player")
            csv_writer.writerow([player_name, team_info["name"], team_info["short_name"]])

    # Print or save the CSV content
    print(csv_output.getvalue())

    # Optionally, save to a file
    with open("bbl_2024-25_players.csv", "w", newline="") as f:
        f.write(csv_output.getvalue())

# Example usage with the input text
input_text = last_line

extract_bbl_2013_14_data(input_text)



Player Name,Team Name,Team Short Name
D&s;Arcy Short,Adelaide Strikers,ADS
Matthew Short,Adelaide Strikers,ADS
Ollie Pope,Adelaide Strikers,ADS
Jake Weatherald,Adelaide Strikers,ADS
James Bazley,Adelaide Strikers,ADS
Cameron Boyce,Adelaide Strikers,ADS
Jordan Buckingham,Adelaide Strikers,ADS
Alex Carey,Adelaide Strikers,ADS
Brendan Doggett,Adelaide Strikers,ADS
Chris Lynn,Adelaide Strikers,ADS
Jamie Overton,Adelaide Strikers,ADS
Lloyd Pope,Adelaide Strikers,ADS
Alex Ross,Adelaide Strikers,ADS
Henry Thornton,Adelaide Strikers,ADS
Harry Nielsen,Adelaide Strikers,ADS
Travis Head,Adelaide Strikers,ADS
Thomas Kelly,Adelaide Strikers,ADS
Liam Scott,Adelaide Strikers,ADS
Harry Manenti,Adelaide Strikers,ADS
Liam Haskett,Adelaide Strikers,ADS
Tom Alsop,Brisbane Heat,BRH
Max Bryant,Brisbane Heat,BRH
Marnus Labuschagne,Brisbane Heat,BRH
Michael Neser,Brisbane Heat,BRH
Jimmy Peirson,Brisbane Heat,BRH
Colin Munro,Brisbane Heat,BRH
Matt Renshaw,Brisbane Heat,BRH
Usman Khawaja,Brisbane Heat,BRH
Will 

In [5]:
import csv

# Sample text response from the webpage
text_response = """
Adelaide Strikers squad
Matt Short (c), Fabian Allen (overseas), James Bazley, Cameron Boyce, Jordan Buckingham, Alex Carey, Brendan Doggett, Travis Head, Tom Kelly, Chris Lynn, Harry Nielsen, Jamie Overton (OS), Lloyd Pope, Ollie Pope (OS), Alex Ross, D’Arcy Short, Henry Thornton, Jake Weatherald.

Brisbane Heat squad
Usman Khawaja (c), Tom Alsop (OS), Xavier Bartlett, Max Bryant, Spencer Johnson, Matt Kuhnemann, Marnus Labuschagne, Nathan McSweeney, Colin Munro (OS), Michael Neser, Jimmy Peirson, Will Prestwidge, Matt Renshaw, Mitchell Swepson, Callum Vidler, Paul Walter (OS), Jack Wildermuth, Jack Wood.

Hobart Hurricanes squad
Nathan Ellis (c), Iain Carlisle, Nikhil Chaudhary, Tim David, Paddy Dooley, Jake Doran, Peter Hatzoglou, Shai Hope (OS), Caleb Jewell, Chris Jordan (OS), Ben McDermott, Riley Meredith, Mitch Owen, Waqar Salamkheil (OS replacement), Billy Stanlake, Matthew Wade, Charlie Wakim, Mac Wright.

Melbourne Renegades squad
Will Sutherland (c), Jacob Bethell (OS), Josh Brown, Xavier Crone, Harry Dixon, Laurie Evans (OS), Jake Fraser-McGurk, Marcus Harris, Mackenzie Harvey, Hassan Khan (OS replacement), Nathan Lyon, Fergus O’Neill, Kane Richardson, Tom Rogers, Gurinder Sandhu, Tim Seifert (OS), Jon Wells, Adam Zampa.

Melbourne Stars squad
Marcus Stoinis (c), Scott Boland, Hilton Cartwright, Joe Clarke (OS replacement), Brody Couch, Tom Curran (OS), Ben Duckett (OS), Sam Harper, Campbell Kellaway, Glenn Maxwell, Hamish McKenzie, Jon Merlo, Adam Milne (OS replacement), Usama Mir (OS), Joel Paris, Tom Rogers, Peter Siddle, Mark Steketee, Doug Warren, Beau Webster.

Perth Scorchers squad
Ashton Turner (c), Ashton Agar, Finn Allen (OS), Mahli Beardman, Jason Behrendorff, Cooper Connolly, Sam Fanning, Aaron Hardie, Nick Hobson, Matthew Hurst (OS), Josh Inglis, Keaton Jennings (OS), Matt Kelly, Mitchell Marsh, Lance Morris, Jhye Richardson, Matthew Spoors, Andrew Tye.

Sydney Sixers squad
Moises Henriques (c), Sean Abbott, Jackson Bird, Jafer Chohan (OS), Joel Davies, Ben Dwarshuis, Jack Edwards, Moises Henriques, Akeal Hosein (OS), Daniel Hughes, Hayden Kerr, Ben Manenti, Todd Murphy, Kurtis Patterson, Mitch Perry, Josh Philippe, Jordan Silk, Steve Smith, James Vince (OS).

Sydney Thunder squad
David Warner (c), Wes Agar, Cameron Bancroft, Sam Billings (OS), Ollie Davies, Lockie Ferguson (OS), Matt Gilkes, Chris Green, Ryan Hadley, Liam Hatcher, Sam Konstas, Nic Maddinson, Nathan McAndrew, Sherfane Rutherford (OS), Will Salzmann, Daniel Sams, Jason Sangha, Tanveer Sangha.
"""

def extract_teams_and_players(text):
    """Extract teams and their players from the given text."""
    teams = {}
    # Split the text into lines
    lines = text.strip().split('\n')
    
    for i in range(0, len(lines), 2):  # Process every two lines (team and players)
        if i + 1 < len(lines):  # Ensure there is a corresponding player line
            team_name = lines[i].replace(" squad", "").strip()  # Get team name
            players = lines[i + 1].strip()  # Get players line
            teams[team_name] = players.split(', ')  # Split players by comma and store in dictionary

    return teams

def write_to_csv(teams, output_file):
    """Write the teams and their players to a CSV file."""
    with open(output_file, mode='w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['Team', 'Players'])  # Write header
        
        for team, players in teams.items():
            writer.writerow([team, ', '.join(players)])  # Write team and players

# Extract teams and players from the text response
teams_and_players = extract_teams_and_players(text_response)

# Write the extracted data to a CSV file
write_to_csv(teams_and_players, 'BBL_Teams_Players.csv')

print("CSV file 'BBL_Teams_Players.csv' created successfully.")

CSV file 'BBL_Teams_Players.csv' created successfully.


In [53]:
import csv
import glob
from collections import defaultdict

def create_master_csv(input_files_pattern, output_file):
    """Create a master CSV file with unique player names and their corresponding teams for specified years."""
    # Specify the years of interest
    years_of_interest = ['2011', '2012', '2013', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
    player_teams = defaultdict(lambda: {year: '' for year in years_of_interest})  # Initialize with empty strings

    # Find all CSV files matching the pattern
    input_files = glob.glob(input_files_pattern)

    # Process each CSV file
    for input_file in input_files:
        # Extract year from the file name (e.g., bbl_2011-12_players)
        year_part = input_file.split('_')[1]  # Get the part after 'bbl_'
        year = year_part.split('-')[0]  # Get the first part of the year (e.g., '2011')

        if year in years_of_interest:  # Only process files for the specified years
            with open(input_file, mode='r') as infile:
                reader = csv.DictReader(infile)
                for row in reader:
                    player_name = row['Player Name']
                    team_name = row['Team Name']
                    player_teams[player_name][year] = team_name  # Assign team name to the player for the corresponding year

    # Create a new CSV file to write the master output
    with open(output_file, mode='w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['Player Name'] + years_of_interest)  # Write header with years

        # Write each unique player and their corresponding teams
        for player, teams in player_teams.items():
            row = [player] + [teams[year] for year in years_of_interest]  # Create row with player name and teams for each year
            writer.writerow(row)  # Write the row to the CSV

# Call the function with the pattern for team files and the output file name
create_master_csv('bbl_*_players.csv', 'Master_Players_Teams.csv')  # Adjust the pattern as needed

In [54]:
import csv

def extract_first_column_to_list(input_file):
    """Extract the first column from the given CSV file and return it as a list."""
    first_column = []

    # Read the original CSV file
    with open(input_file, mode='r') as infile:
        reader = csv.reader(infile)
        next(reader)  # Skip the header row if there is one

        # Collect values from the first column
        for row in reader:
            if row:  # Ensure the row is not empty
                first_column.append(row[0])  # Append the first column value

    return first_column

# Example usage
csv_file = 'data/Final_Players_YearWise_IPL_data.csv'  # Replace with your actual CSV file name
first_column_list = extract_first_column_to_list(csv_file)
print(first_column_list)  # Print the list of values from the first column

['Gurnoor Singh Brar', 'Nishant Sindhu', 'Karim Janat', 'Lockie Ferguson', 'Shubman Gill', 'Prasidh Krishna', 'Abhinav Manohar', 'Shahrukh Khan', 'Rahmanullah Gurbaz', 'Sai Kishore', 'Noor Ahmad', 'Azmatullah Omarzai', 'Washington Sundar', 'Glenn Phillips', 'Yash Dayal', 'Vijay Shankar', 'Kulwant Khejroliya', 'Kumar Kushagra', 'Mohammed Shami', 'Sai Sudharsan', 'Spencer Johnson', 'Rahul Tewatia', 'Hardik Pandya', 'David Miller', 'Gerald Coetzee', 'Darshan Nalkande', 'Rashid Khan', 'Jayant Yadav', 'Manav Suthar', 'Anuj Rawat', 'Sherfane Rutherford', 'Arshad Khan', 'Kagiso Rabada', 'Mohammed Siraj', 'Ishant Sharma', 'Mahipal Lomror', 'Mohit Sharma', 'Jos Buttler', 'Rinku Singh', 'Arshdeep Singh', 'Ravichandran Ashwin', 'Atharva Taide', 'Sandeep Sharma', 'Yuzvendra Chahal', 'Karun Nair', 'Varun Chakaravarthy', 'Vaibhav Arora', 'Marco Jansen', 'Shashank Singh', 'Deepak Hooda', 'Pyla Avinash', 'Vishnu Vinod', 'Mohit Rathee', 'Nehal Wadhera', 'Aiden Markram', 'KL Rahul', 'Harpreet Brar', 'Xa

In [56]:
import csv

def filter_players(input_file, output_file, player_list):
    """Filter players from the master CSV file based on the provided list."""
    # Convert the player list to a set for faster lookup
    player_set = set(player_list)

    with open(input_file, mode='r') as infile:
        reader = csv.DictReader(infile)
        filtered_players = []

        # Process each row
        for row in reader:
            player_name = row['Player Name']
            if player_name in player_set:  # Check if the player is in the provided list
                filtered_players.append(row)  # Add the row to the filtered list

    # Create a new CSV file to write the filtered output
    with open(output_file, mode='w', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)  # Use the same fieldnames as the input
        writer.writeheader()  # Write header
        writer.writerows(filtered_players)  # Write the filtered rows

# Example usage
input_file = 'Master_Players_Teams.csv'  # The master CSV file
output_file = 'Filtered_Players_Teams.csv'  # The output file for filtered players

# List of players to keep (example)
player_list = [
    'Brendon McCullum',
    'Matthew Arnold',
    'James Hopes',
    # Add more player names as needed
]

# Call the function to filter players
filter_players(input_file, output_file, first_column_list)

print(f"Filtered CSV file '{output_file}' created successfully.")

Filtered CSV file 'Filtered_Players_Teams.csv' created successfully.
